In [1]:
import numpy as np
from scipy import io
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import itertools
from joblib import Parallel, delayed
import multiprocessing
import operator
from utils import train_test_indexes
import time
import pickle
import send_mail as mail
import itertools
from markov_chains import *

%matplotlib inline
import mpld3
#mpld3.enable_notebook()

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

# Test random_rankings avec build_fonction8 seulement

In [2]:
train = io.loadmat('/home/max/projects/challengeMDI343/data/data_train.mat')

probes = { train['probeId'].ravel()[i]: train['probeLabel'].ravel()[i] for i in range(len(train['probeLabel']))}
gallery = {train['galleryId'].ravel()[i]: train['galleryLabel'].ravel()[i] for i in range(len(train['galleryLabel']))}

In [23]:
train_ind, test_ind = train_test_indexes(len(train['probeId']), test_size=0.99, seed=1)

print train_ind.shape, test_ind.shape

predId = train['resultsId'][train_ind, :, :30]
probeId = train['probeId'][train_ind, 0]

(474,) (46889,)


In [27]:
def compute_bestId(ranks, probeId, i):
    ranks = random_rankings(ranks, n_estimators=3, min_features=2, max_features=8, min_items=4, max_items=None, 
                            eps_min=0.001, eps_max=0.3, 
                            seed=10 ,n_jobs=None)
    uniques, count = np.unique(ranks[0,:], return_counts=True)
    bestId = uniques[np.argmax(count)]
    
    return {probeId: bestId}
    

n_jobs = multiprocessing.cpu_count()
        
t0 = time.time()
results = Parallel(n_jobs=n_jobs-2)(delayed(compute_bestId)(predId[i,:,:], probeId[i], i) for i in range(len(predId)))

chron = (time.time() - t0)/60.
print 'time:', chron

results = { k.keys()[0]: k.values()[0] for k in results }

#with open('result_random_rankings_model0_n20_maxf8_train.pkl', 'w') as fp:
#    pickle.dump(results, fp)
    
#mail.sendMail('maxime.kubryk@gmail.com','script markov-chain all train ended', 'ended in '+str(chron)+' min')


time: 0.31388275226


In [28]:
#with open('result_random_rankings_model0_n20_maxf8_train.pkl', 'rb') as fp:
#    results = pickle.load(fp)

print results

#train = io.loadmat('/home/max/projects/challengeMDI343/data/data_train.mat')
#gallery = {train['galleryId'].ravel()[i]: train['galleryLabel'].ravel()[i] for i in range(len(train['galleryLabel']))}
#probeId = train['probeId'][:, 0]
probeLabel = train['probeLabel'][train_ind, 0]

preds = np.asarray([int(gallery[results[i]]) for i in probeId ])

accuracy = np.mean(preds == probeLabel)

print
print 'accuracy', accuracy

{59392: 98280, 4104: 28010, 106516: 96916, 47128: 36705, 45081: 144601, 133155: 110877, 84004: 62460, 4133: 59707, 40: 26624, 157739: 51342, 55342: 45534, 36928: 117278, 127043: 53270, 82274: 147914, 67663: 107459, 57437: 91540, 80229: 77552, 102497: 6207, 57442: 18021, 114640: 103408, 6250: 136576, 108654: 81057, 57457: 142455, 157374: 25032, 94332: 100555, 114815: 110386, 135297: 80392, 26760: 60948, 125067: 34520, 127117: 15989, 151696: 120751, 73873: 27690, 12441: 71424, 131231: 60615, 67751: 55306, 2217: 5436, 80047: 85298, 32944: 147223, 88242: 147925, 80053: 149193, 22717: 118663, 65726: 35429, 43205: 134353, 153798: 153891, 80071: 67799, 135374: 8330, 147666: 59800, 121051: 32253, 153822: 76955, 102437: 1518, 141545: 90545, 49388: 6952, 138280: 62662, 45300: 30231, 133374: 75073, 24833: 51503, 65799: 12506, 92439: 117840, 112921: 94241, 12570: 14284, 131357: 81095, 80158: 39200, 135463: 109824, 125232: 19751, 41265: 91864, 86327: 93610, 35130: 117380, 90429: 97823, 102724: 1290

- random_rankings(ranks, n_estimators=10, max_features=8, max_eps=0.8, seed=10 ,n_jobs=None)
  - train_test seed = 42
  - predId = train['resultsId'][train_ind, :, :10]
  - test_size=0.95 : 2369
  - ==> accuracy = 0.975094976783
  
- random_rankings(ranks, n_estimators=10, max_features=8, max_eps=0.8, seed=10 ,n_jobs=None)
  - train_test seed = 1
  - predId = train['resultsId'][train_ind, :, :10]
  - test_size=0.95 : 2369
  - ==> accuracy = 0.97214014352
  
- random_rankings(ranks, n_estimators=10, max_features=8, max_eps=0.8, seed=10 ,n_jobs=None)
  - train_test seed = 42
  - predId = train['resultsId'][train_ind, :, :15]
  - test_size=0.95 : 2369
  - ==> accuracy = 0.979738286197
  
- random_rankings(ranks, n_estimators=10, max_features=8, max_eps=0.8, seed=10 ,n_jobs=None)
  - train_test seed = 1
  - predId = train['resultsId'][train_ind, :, :15]
  - test_size=0.95 : 2369
  - ==> accuracy = 0.973828619671